# Lotte Data

## Preprocessing

In [14]:
!pip install --force https://github.com/chengs/tqdm/archive/colab.zip

     \ 471kB 11.7MB/s
  Running setup.py bdist_wheel for tqdm ... - \ done
  Stored in directory: /tmp/pip-ephem-wheel-cache-te1d890j/wheels/41/18/ee/d5dd158441b27965855b1bbae03fa2d8a91fe645c01b419896
Successfully built tqdm
spacy 2.0.18 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
pymc3 3.6 has requirement joblib<0.13.0, but you'll have joblib 0.13.0 which is incompatible.
featuretools 0.4.1 has requirement pandas>=0.23.0, but you'll have pandas 0.22.0 which is incompatible.
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


In [15]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs
from tqdm import tqdm_notebook as tqdm
import json
from google.colab import drive 
import urllib.request
from pandas.io.json import json_normalize
drive.mount('/content/gdrive')



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [16]:
prod = pd.read_csv('gdrive/My Drive/공모전/Lpoint/data/Pruduct.csv')
mast = pd.read_csv('gdrive/My Drive/공모전/Lpoint/data/Master.csv')
sess = pd.read_csv('gdrive/My Drive/공모전/Lpoint/data/Session.csv')
cust = pd.read_csv('gdrive/My Drive/공모전/Lpoint/data/Custom.csv')
# sear = pd.read_csv('gdrive/My Drive/공모전/Lpoint/data/Search1.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#### 자료형 변환
- **object** to **integer**

In [17]:
prod['PD_BUY_AM'] = pd.to_numeric(prod['PD_BUY_AM'].str.replace(',',''))
prod['PD_BUY_CT'][29649] = 1440
prod['PD_BUY_CT'][34105] = 1387
prod['PD_BUY_CT'] = pd.to_numeric(prod['PD_BUY_CT'])

sess['TOT_SESS_HR_V'] = pd.to_numeric(sess['TOT_SESS_HR_V'].str.replace(',',''))
# sess['SESS_HR_PAG_VIEW'] = sess['TOT_SESS_HR_V'] / sess['TOT_PAG_VIEW_CT']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


#### 브랜드명 수정
- 브랜드명에서 **불필요한 단어** 제거(예. 향수, 괄호, 공백 등)
- **잘못입력된** 브랜드명 수정

In [0]:
prod['PD_BRA_NM'] = prod['PD_BRA_NM'].map(lambda x: x.split('(')[0].replace('[','').replace(']','').replace(' ',''))
prod.loc[prod['PD_BRA_NM']=='MADE IN ITALY','PD_BRA_NM'] = 'V73'
prod.loc[prod['PD_BRA_NM'].isin(['1909535','19330640825']),'PD_BRA_NM'] = 'KL'
prod.loc[prod['PD_BRA_NM']=='19386640906','PD_BRA_NM'] = '지이크'
prod.loc[prod['PD_BRA_NM'].isin(['1907517','1924113','1906469','1906471','1902295','1908178','1881326','1924116','1906492','1902287']),'PD_BRA_NM'] = 'CC collect'
prod.loc[prod['PD_BRA_NM'].isin(['19379580904','19364580830','1872892','1898551','18703070925','19359410830']),'PD_BRA_NM'] = '더아이잗'
prod.loc[prod['PD_BRA_NM'].isin(['1904418','1896342']),'PD_BRA_NM'] = '보니스팍스'
prod.loc[prod['PD_BRA_NM'].isin(['1912506','1912518']),'PD_BRA_NM'] = '리스트'
prod.loc[prod['PD_BRA_NM']=='19326280823','PD_BRA_NM'] = 'ENC'
prod.loc[prod['PD_BRA_NM'].isin(['19266920905','19266920905','19266920905']),'PD_BRA_NM'] = '요하넥스'
prod.loc[prod['PD_BRA_NM']=='1913802','PD_BRA_NM'] = '꼼빠니아'
prod.loc[prod['PD_BRA_NM']=='1917759','PD_BRA_NM'] = '인디안'

In [0]:
prod_mast = pd.merge(prod, mast, how = 'inner', on=['PD_C'])


# df[df['PD_BUY_CT']>=150000].sort_values(by=['PD_BUY_AM'],ascending=False)
# cate_list= ['패션잡화','여성의류','화장품/뷰티케어','남성의류']
cate_list= ['패션잡화','여성의류','유아동의류','스포츠패션','남성의류']
prod_mast = prod_mast[prod_mast['CLAC1_NM'].isin(cate_list)]

In [0]:
# l = ['18703070925','1872892','1881326','1896342','1898551','1902287','1902295','1904418','1906469','1906471','1906492','1907517','1908178',
#      '1912506','1912518','1913802', '1917759','1924113','1924116','19266920905','19326280823','19359410830','19364580830','19379580904']
# prod_mast[prod_mast['PD_BRA_NM'].isin(l)]

#### 명품브랜드만 골라내기

In [21]:
mp1 = pd.read_csv("gdrive/My Drive/공모전/Lpoint/명품 골라내기/lotte_luxury_list.csv")
brand_lotte = list(mp1['BRA_NM'])
len(brand_lotte)

275

In [22]:
mp2 = pd.read_csv('gdrive/My Drive/공모전/Lpoint/명품 골라내기/my_luxury_list2.csv')
brands = brand_lotte
brands.extend(mp2['title'])
brands = list(set(brands))
len(brands)

426

In [23]:
prod_mast = prod_mast[prod_mast['PD_BRA_NM'].isin(brands)]
prod_mast.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 311019 entries, 3984 to 5024890
Data columns (total 12 columns):
CLNT_ID      311019 non-null int64
SESS_ID      311019 non-null int64
HITS_SEQ     311019 non-null int64
PD_C         311019 non-null int64
PD_ADD_NM    311019 non-null object
PD_BRA_NM    311019 non-null object
PD_BUY_AM    311019 non-null int64
PD_BUY_CT    311019 non-null int64
PD_NM        311019 non-null object
CLAC1_NM     311019 non-null object
CLAC2_NM     311019 non-null object
CLAC3_NM     311019 non-null object
dtypes: int64(6), object(6)
memory usage: 30.8+ MB


### Merge data

In [0]:
pmc = pd.merge(prod_mast, cust, how = 'left', on=['CLNT_ID'])
pms = pd.merge(prod_mast, sess, how = 'inner', on=['CLNT_ID','SESS_ID'])

## By Product

### Price

In [37]:
df_pm = prod_mast.groupby(['CLAC1_NM','CLAC2_NM','CLAC3_NM','PD_BRA_NM']).agg({'PD_BUY_AM':'mean','PD_BUY_CT':'sum'}).reset_index()
df_pm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2898 entries, 0 to 2897
Data columns (total 6 columns):
CLAC1_NM     2898 non-null object
CLAC2_NM     2898 non-null object
CLAC3_NM     2898 non-null object
PD_BRA_NM    2898 non-null object
PD_BUY_AM    2898 non-null float64
PD_BUY_CT    2898 non-null int64
dtypes: float64(1), int64(1), object(4)
memory usage: 135.9+ KB


In [38]:
df_pm['PD_BUY_AM'] = round(df_pm['PD_BUY_AM']).map(lambda x: int(x))
df_pm.head()

,CLAC1_NM,CLAC2_NM,CLAC3_NM,PD_BRA_NM,PD_BUY_AM,PD_BUY_CT
0,남성의류,남성의류상의,남성가디건,갤럭시,239200,1
1,남성의류,남성의류상의,남성가디건,게스,63146,144
2,남성의류,남성의류상의,남성가디건,꼼데가르송,466000,2
3,남성의류,남성의류상의,남성가디건,내셔널지오그래픽,84257,7
4,남성의류,남성의류상의,남성가디건,라코스테,248476,68


### Sales

In [39]:
df_pm['PD_SALES'] = df_pm['PD_BUY_AM']*df_pm['PD_BUY_CT']
df_pm.head()

,CLAC1_NM,CLAC2_NM,CLAC3_NM,PD_BRA_NM,PD_BUY_AM,PD_BUY_CT,PD_SALES
0,남성의류,남성의류상의,남성가디건,갤럭시,239200,1,239200
1,남성의류,남성의류상의,남성가디건,게스,63146,144,9093024
2,남성의류,남성의류상의,남성가디건,꼼데가르송,466000,2,932000
3,남성의류,남성의류상의,남성가디건,내셔널지오그래픽,84257,7,589799
4,남성의류,남성의류상의,남성가디건,라코스테,248476,68,16896368


## By Customer


In [40]:
pmc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 311019 entries, 0 to 311018
Data columns (total 15 columns):
CLNT_ID        311019 non-null int64
SESS_ID        311019 non-null int64
HITS_SEQ       311019 non-null int64
PD_C           311019 non-null int64
PD_ADD_NM      311019 non-null object
PD_BRA_NM      311019 non-null object
PD_BUY_AM      311019 non-null int64
PD_BUY_CT      311019 non-null int64
PD_NM          311019 non-null object
CLAC1_NM       311019 non-null object
CLAC2_NM       311019 non-null object
CLAC3_NM       311019 non-null object
CLNT_GENDER    250056 non-null object
CLNT_AGE       250056 non-null float64
GENDER_AGE     311019 non-null object
dtypes: float64(1), int64(6), object(8)
memory usage: 38.0+ MB


### Gender & Age

In [0]:
pmc['GENDER_AGE'] = pmc[['CLNT_GENDER','CLNT_AGE']].apply(lambda x: '{}_{}'.format(x[0],x[1]), axis=1)

In [42]:
g_pmc = pmc.groupby(['CLAC1_NM','CLAC2_NM','CLAC3_NM','PD_BRA_NM','GENDER_AGE']).agg({'CLNT_ID':'count'}).reset_index()
g_pmc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13738 entries, 0 to 13737
Data columns (total 6 columns):
CLAC1_NM      13738 non-null object
CLAC2_NM      13738 non-null object
CLAC3_NM      13738 non-null object
PD_BRA_NM     13738 non-null object
GENDER_AGE    13738 non-null object
CLNT_ID       13738 non-null int64
dtypes: int64(1), object(5)
memory usage: 644.0+ KB


In [43]:
g_pmc.head()

,CLAC1_NM,CLAC2_NM,CLAC3_NM,PD_BRA_NM,GENDER_AGE,CLNT_ID
0,남성의류,남성의류상의,남성가디건,갤럭시,M_50.0,1
1,남성의류,남성의류상의,남성가디건,게스,F_20.0,3
2,남성의류,남성의류상의,남성가디건,게스,F_30.0,22
3,남성의류,남성의류상의,남성가디건,게스,F_40.0,72
4,남성의류,남성의류상의,남성가디건,게스,F_50.0,22


In [44]:
df_pmc = g_pmc.pivot_table(values='CLNT_ID',index=['CLAC1_NM','CLAC2_NM','CLAC3_NM','PD_BRA_NM'],columns='GENDER_AGE').fillna(0).reset_index()
df_pmc.head()

GENDER_AGE,CLAC1_NM,CLAC2_NM,CLAC3_NM,PD_BRA_NM,F_10.0,F_20.0,F_30.0,F_40.0,F_50.0,F_60.0,F_70.0,M_10.0,M_20.0,M_30.0,M_40.0,M_50.0,M_60.0,M_70.0,M_80.0,nan_nan
0,남성의류,남성의류상의,남성가디건,갤럭시,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,남성의류,남성의류상의,남성가디건,게스,0.0,3.0,22.0,72.0,22.0,0.0,0.0,0.0,0.0,3.0,2.0,1.0,0.0,0.0,0.0,17.0
2,남성의류,남성의류상의,남성가디건,꼼데가르송,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,남성의류,남성의류상의,남성가디건,내셔널지오그래픽,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0
4,남성의류,남성의류상의,남성가디건,라코스테,0.0,10.0,17.0,13.0,6.0,0.0,0.0,2.0,7.0,1.0,3.0,4.0,0.0,0.0,0.0,5.0


### Sign in

In [36]:
df_pmc.columns = ['CLAC1_NM','CLAC2_NM','CLAC3_NM', 'PD_BRA_NM', 'F_10', 'F_20', 'F_30', 'F_40',
       'F_50', 'F_60', 'F_70', 'F_80', 'M_10', 'M_20', 'M_30',
       'M_40', 'M_50', 'M_60', 'M_70', 'M_80', 'NOT_SIGNED']
df_pmc.head()

ValueError: ignored

## By Session

In [0]:
pms.info()

### Location

In [0]:
g_pms_l = pms.groupby(['CLAC1_NM','CLAC2_NM','CLAC3_NM','PD_BRA_NM','ZON_NM']).agg({'CLNT_ID':'count'}).reset_index()
g_pms_l.info()

In [0]:
df_pms_l = g_pms_l.pivot_table(values='CLNT_ID',index=['CLAC1_NM','CLAC2_NM','CLAC3_NM','PD_BRA_NM'],columns='ZON_NM').fillna(0).reset_index()
df_pms_l.head()

In [0]:
df_pms_l.columns = ['CLAC1_NM','CLAC2_NM','CLAC3_NM', 'PD_BRA_NM', 'Busan', 'Chungbuk',
       'Chungnam', 'Daegu', 'Daejeon', 'Gangwon', 'Gwangju',
       'Gyeonggi', 'Gyeongbu', 'Gyeongnam', 'Incheon',
       'Jeju', 'Jeonbuk', 'Jeonnam', 'Seoul', 'Ulsan']
df_pms_l.head()

### Device

In [0]:
g_pms_d = pms.groupby(['CLAC1_NM','CLAC2_NM','CLAC3_NM','PD_BRA_NM','DVC_CTG_NM']).agg({'CLNT_ID':'count'}).reset_index()
g_pms_d.loc[g_pms_d['DVC_CTG_NM'] == 'tablet','DVC_CTG_NM'] = 'mobile'
g_pms_d.info()

In [0]:
df_pms_d = g_pms_d.pivot_table(values='CLNT_ID',index=['CLAC1_NM','CLAC2_NM','CLAC3_NM','PD_BRA_NM'],columns='DVC_CTG_NM').fillna(0).reset_index()
df_pms_d.head()

In [0]:
df_pms_d.columns = ['CLAC1_NM','CLAC2_NM','CLAC3_NM', 'PD_BRA_NM', 'Desktop', 'Mobile']
df_pms_d.head()

### Merge

In [0]:
df_pms = pd.merge(df_pms_d,df_pms_l, how='inner', on=['CLAC1_NM','CLAC2_NM','CLAC3_NM','PD_BRA_NM'])
df_pms.head()

## Merge ALL

In [0]:
data =  pd.merge(df_pm,df_pmc, how='inner', on=['CLAC1_NM','CLAC2_NM','CLAC3_NM','PD_BRA_NM'])
data =  pd.merge(data,df_pms, how='inner', on=['CLAC1_NM','CLAC2_NM','CLAC3_NM','PD_BRA_NM'])
data.info()

In [0]:
data.to_csv('gdrive/My Drive/공모전/Lpoint/데이터셋 만들기/lotte.csv',index=False)